In [5]:
# 在 Notebook 裡跑
import sys
!{sys.executable} -m pip install -U ipywidgets jupyterlab_widgets widgetsnbextension

   ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
   -- ------------------------------------- 10.2/139.8 kB ? eta -:--:--
   -------------------------------- ------- 112.6/139.8 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 139.8/139.8 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/216.6 kB ? eta -:--:--
   ---------------------------------------- 216.6/216.6 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ------------- -------------------------- 0.8/2.2 MB 16.1 MB/s eta 0:00:01
   ----------------------------------- ---- 1.9/2.2 MB 20.4 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 15.5 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.5.2
    Uninstalling widgetsnbextension-3.5.2:
      Successfully uninstalled widgetsnbextension-3.5.2
  Attempting uninstall: jupyterlab_widgets
   

In [2]:
import ipywidgets, jupyterlab_widgets, widgetsnbextension
print("ipywidgets:", ipywidgets.__version__)
print("jupyterlab_widgets:", jupyterlab_widgets.__version__)
print("widgetsnbextension:", widgetsnbextension.__version__)

ipywidgets: 8.1.7
jupyterlab_widgets: 3.0.15
widgetsnbextension: 4.0.14


In [1]:
import ipywidgets as W
from IPython.display import display

display(W.HTML("<b>ipywidgets OK</b>"))
display(W.IntSlider(value=3, min=0, max=10, description="測試滑桿"))

HTML(value='<b>ipywidgets OK</b>')

IntSlider(value=3, description='測試滑桿', max=10)

In [2]:
# --- 麥當勞點餐機 (ipywidgets) — 完整版 / PNG 圖片版 ---
# 依據規格書 v2.1（點餐機 + 購物車內份數修改）
# 需求：ipywidgets、Pillow
# pip install ipywidgets pillow

from IPython.display import display
import ipywidgets as W
from datetime import datetime
import re, os, io
from PIL import Image

# ========= 參數與資料 =========

# 價格表（同類內不重複）
PRICES_MAIN = {
    "bigmac": 89, "mcchicken": 79, "filetofish": 95, "dblcheese": 99,
}
PRICES_FRIES = {
    "fries_s": 35, "fries_m": 45, "fries_l": 55,
}
PRICES_NUGGETS = {
    "nuggets4": 49, "nuggets6": 65, "nuggets10": 109,
}
PRICES_SNACK_OTHER = {
    "apple_pie": 42, "corn_soup": 55, "side_salad": 50, "sundae_choco": 45,
}
PRICES_DRINK = {
    "coke": 38, "sprite": 36, "iced_tea": 32, "hot_coffee": 40,
}

# 中文名稱映射
NAME_MAIN = {
    "bigmac": "大麥克", "mcchicken": "麥香雞", "filetofish": "麥香魚", "dblcheese": "雙層牛肉吉事堡",
}
NAME_FRIES_SIZE = {"fries_s": "小薯", "fries_m": "中薯", "fries_l": "大薯"}
NAME_FRIES_SALT = {"nosalt": "無鹽", "salt": "有鹽"}
NAME_NUGGETS_SIZE = {"nuggets4": "4塊", "nuggets6": "6塊", "nuggets10": "10塊"}
NAME_NUGGETS_SAUCE = {"none": "不需要醬料", "sweet_chili": "甜辣醬", "mustard": "芥末醬", "bbq": "BBQ醬"}
NAME_SNACK_OTHER = {
    "apple_pie": "蘋果派", "corn_soup": "玉米濃湯", "side_salad": "田園沙拉", "sundae_choco": "巧克力聖代",
}
NAME_DRINK = {
    "coke": "可口可樂(去冰)", "sprite": "雪碧", "iced_tea": "冰紅茶", "hot_coffee": "熱美式",
}

# 圖片對應（檔案請放在 ./assets/，若無則顯示 none.png）
IMG_MAP = {
    # 主餐
    "bigmac": "bigmac.png", "mcchicken": "mcchicken.png",
    "filetofish": "filetofish.png", "dblcheese": "dblcheese.png",
    # 薯條（以尺寸為主，口味不分圖）
    "fries_s": "fries_s.png", "fries_m": "fries_m.png", "fries_l": "fries_l.png",
    # 雞塊（以份量為主）
    "nuggets4": "nuggets4.png", "nuggets6": "nuggets6.png", "nuggets10": "nuggets10.png",
    # 其他點心
    "apple_pie": "apple_pie.png", "corn_soup": "corn_soup.png",
    "side_salad": "side_salad.png", "sundae_choco": "sundae_choco.png",
    # 飲料
    "coke": "coke.png", "sprite": "sprite.png", "iced_tea": "iced_tea.png", "hot_coffee": "hot_coffee.png",
}
ASSETS_DIR = "./assets"
DEFAULT_IMG = "none.png"

# 訂單序號（Notebook 期間累計）
ORDER_COUNTER = {"n": 0}

# 購物車：sku_key -> {name_display, unit_price, qty, category}
CART = {}

# ========= 小工具：載圖與工具函式 =========
def load_image_bytes(key: str) -> bytes:
    """根據品項 key 載入圖片；若不存在則顯示 DEFAULT_IMG。"""
    filename = IMG_MAP.get(key, DEFAULT_IMG)
    path = os.path.join(ASSETS_DIR, filename)
    if not os.path.exists(path):
        path = os.path.join(ASSETS_DIR, DEFAULT_IMG)
    try:
        with Image.open(path) as im:
            im = im.convert("RGBA")  # 保留透明度
            bio = io.BytesIO()
            im.save(bio, format="PNG")   # 以 PNG 輸出到記憶體
            return bio.getvalue()
    except Exception:
        return b""

def currency(n: int) -> str:
    return f"${n:,}"

def next_order_id() -> str:
    ORDER_COUNTER["n"] += 1
    seq = f"{ORDER_COUNTER['n']:03d}"
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    return f"MCD-{ts}-{seq}"

def tw_tax_id_ok(b: str) -> bool:
    # 簡易檢查：8 位數字（進階校驗可自行擴充）
    return bool(re.fullmatch(r"\d{8}", b))

def carrier_hint_ok(c: str) -> bool:
    # 常見手機條碼：/ 開頭 + 7 碼英數 . + -
    return bool(re.fullmatch(r"/[A-Z0-9.+-]{7}", c.upper()))

# ========= 版面骨架 =========
title = W.HTML("<h2>🍔 麥當勞點餐機（ipywidgets / PNG）</h2>")

# 左側：類別切換
tb_category = W.ToggleButtons(
    options=[("主餐", "main"), ("點心", "snack"), ("飲料", "drink")],
    value="main",
    description="類別",
)

# 中間：商品卡容器（由類別動態重建）
product_area = W.VBox()

# 右側：小計 + 圖片預覽 + 購物車 + 結帳面板
lbl_subtotal = W.HTML("<b>小計：$0</b>")
img_preview = W.Image(value=load_image_bytes("bigmac"), format="png", width=320, height=240)
cart_area = W.VBox()           # 動態購物車表格
right_panel = W.VBox()

# 結帳面板 (Accordion)
cb_bag = W.Checkbox(value=False, description="需要購物袋 (+1)")
txt_tax_id = W.Text(value="", placeholder="8 碼數字，例如 12345678", description="統編")
txt_carrier = W.Text(value="", placeholder="/AB12345（手機條碼，可留白）", description="載具")
btn_checkout = W.Button(description="產生收據", button_style="success", icon="credit-card")
out_receipt = W.Output()

checkout_panel = W.VBox([cb_bag, txt_tax_id, txt_carrier, btn_checkout, out_receipt])
acc_checkout = W.Accordion(children=[checkout_panel])
acc_checkout.set_title(0, "結帳")

# 整體三欄
left_col = W.VBox([tb_category])
mid_col = W.VBox([product_area])
right_col = W.VBox([lbl_subtotal, img_preview, W.HTML("<hr>"), W.HTML("<b>購物車</b>"), cart_area, W.HTML("<hr>"), acc_checkout])

ui = W.HBox([left_col, mid_col, right_col],
            layout=W.Layout(width="100%", justify_content="space-between", align_items="flex-start", gap="24px"))

# ========= 購物車渲染與金額 =========
def compute_subtotal() -> int:
    return sum(item["unit_price"] * item["qty"] for item in CART.values()) + (1 if cb_bag.value else 0)

def update_subtotal_label():
    sub = compute_subtotal()
    lbl_subtotal.value = f"<b>小計：{currency(sub)}</b>"

def remove_cart_item(sku: str):
    if sku in CART:
        del CART[sku]
    render_cart()

def on_qty_changed(change, sku: str):
    if change["name"] == "value":
        try:
            newq = int(change["new"])
        except Exception:
            newq = 0
        newq = max(0, newq)
        if newq == 0:
            remove_cart_item(sku)
        else:
            CART[sku]["qty"] = newq
            render_cart()

def render_cart():
    # 重新建構購物車表格
    rows = []
    if not CART:
        rows.append(W.HTML("<i>購物車目前是空的</i>"))
    else:
        # 表頭
        header = W.HBox([
            W.HTML("<b>品項</b>", layout=W.Layout(width="55%")),
            W.HTML("<b>單價</b>", layout=W.Layout(width="12%")),
            W.HTML("<b>份數</b>", layout=W.Layout(width="16%")),
            W.HTML("<b>小計</b>", layout=W.Layout(width="12%")),
            W.HTML("", layout=W.Layout(width="5%")),
        ])
        rows.append(header)

        for sku, item in CART.items():
            name = item["name_display"]
            unit = item["unit_price"]
            qty = item["qty"]
            line = unit * qty

            name_w = W.HTML(name, layout=W.Layout(width="55%"))
            unit_w = W.Label(currency(unit), layout=W.Layout(width="12%"))
            qty_w = W.IntText(value=qty, min=0, step=1, layout=W.Layout(width="90%"))
            qty_w.observe(lambda ch, s=sku: on_qty_changed(ch, s), names="value")

            sub_w = W.Label(currency(line), layout=W.Layout(width="12%"))
            btn_del = W.Button(icon="trash", tooltip="刪除", layout=W.Layout(width="32px"))
            btn_del.on_click(lambda b, s=sku: remove_cart_item(s))

            row = W.HBox([name_w, unit_w, W.HBox([qty_w], layout=W.Layout(width="16%")), sub_w, btn_del])
            rows.append(row)

    cart_area.children = rows
    update_subtotal_label()

# ========= 預覽圖 =========
def set_preview(key: str):
    img_preview.value = load_image_bytes(key)

# ========= 加入購物車 =========
def add_to_cart(sku_key: str, name_display: str, unit_price: int, qty: int, category: str):
    if qty <= 0:
        return  # 不加入 0 份
    if sku_key in CART:
        CART[sku_key]["qty"] += qty
    else:
        CART[sku_key] = {
            "name_display": name_display,
            "unit_price": unit_price,
            "qty": qty,
            "category": category,
        }
    render_cart()

# ========= 中間商品卡：依類別切換 =========
status_msg = W.HTML("")

def build_main_card():
    dd_item = W.Dropdown(
        options=[(NAME_MAIN[k], k) for k in PRICES_MAIN.keys()],
        value="bigmac", description="主餐"
    )
    qty = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add = W.Button(description="加入購物車", button_style="primary", icon="plus")
    status_msg.value = ""

    def _on_main_change(change):
        if change["name"] == "value":
            set_preview(change["new"])

    dd_item.observe(_on_main_change, names="value")

    def _add(_):
        pid = dd_item.value
        name = NAME_MAIN[pid]
        price = PRICES_MAIN[pid]
        add_to_cart(pid, name, price, qty.value, "main")
        status_msg.value = f"<span style='color:green'>已加入：{name} × {qty.value}</span>"

    btn_add.on_click(_add)

    set_preview(dd_item.value)
    return W.VBox([W.HBox([dd_item, qty, btn_add]), status_msg])

def build_snack_card():
    # 點心分三類：薯條 / 麥克雞塊 / 其他點心
    dd_kind = W.ToggleButtons(options=[("薯條", "fries"), ("麥克雞塊", "nuggets"), ("其他點心", "other")],
                              value="fries", description="點心類型")

    # --- 薯條卡 ---
    dd_fries_size = W.Dropdown(options=[(NAME_FRIES_SIZE[k], k) for k in PRICES_FRIES.keys()],
                               value="fries_m", description="尺寸")
    dd_fries_salt = W.Dropdown(options=[("無鹽", "nosalt"), ("有鹽", "salt")], value="salt", description="口味")
    qty_fries = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add_fries = W.Button(description="加入購物車", button_style="primary", icon="plus")

    def on_fries_change(change):
        if change["name"] == "value":
            set_preview(dd_fries_size.value)
    dd_fries_size.observe(on_fries_change, names="value")
    set_preview(dd_fries_size.value)

    def add_fries(_):
        size = dd_fries_size.value
        salt = dd_fries_salt.value
        name = f"{NAME_FRIES_SIZE[size]}（{NAME_FRIES_SALT[salt]}）"
        price = PRICES_FRIES[size]
        sku = f"{size}|{salt}"
        add_to_cart(sku, name, price, qty_fries.value, "snack")
        status_msg.value = f"<span style='color:green'>已加入：{name} × {qty_fries.value}</span>"
    btn_add_fries.on_click(add_fries)
    fries_card = W.VBox([W.HBox([dd_fries_size, dd_fries_salt]), W.HBox([qty_fries, btn_add_fries])])

    # --- 麥克雞塊卡 ---
    dd_n_size = W.Dropdown(options=[(NAME_NUGGETS_SIZE[k], k) for k in PRICES_NUGGETS.keys()],
                           value="nuggets6", description="份量")
    cb_need_sauce = W.Checkbox(value=True, description="需要醬料")
    dd_n_sauce = W.Dropdown(options=[("不需要", "none"), ("甜辣醬", "sweet_chili"), ("芥末醬", "mustard"), ("BBQ醬", "bbq")],
                            value="sweet_chili", description="醬料")
    qty_n = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add_n = W.Button(description="加入購物車", button_style="primary", icon="plus")

    def on_n_size_change(change):
        if change["name"] == "value":
            set_preview(dd_n_size.value)
    dd_n_size.observe(on_n_size_change, names="value")
    set_preview(dd_n_size.value)

    def on_cb_need(change):
        need = cb_need_sauce.value
        dd_n_sauce.disabled = not need
        if not need:
            dd_n_sauce.value = "none"
    cb_need_sauce.observe(on_cb_need, names="value")
    on_cb_need(None)

    def add_n(_):
        nsize = dd_n_size.value
        sauce = dd_n_sauce.value
        price = PRICES_NUGGETS[nsize]
        sauce_name = NAME_NUGGETS_SAUCE[sauce] if cb_need_sauce.value else NAME_NUGGETS_SAUCE["none"]
        name = f"麥克雞塊 {NAME_NUGGETS_SIZE[nsize]}（{sauce_name}）"
        sku = f"{nsize}|{sauce if cb_need_sauce.value else 'none'}"
        add_to_cart(sku, name, price, qty_n.value, "snack")
        status_msg.value = f"<span style='color:green'>已加入：{name} × {qty_n.value}</span>"
    btn_add_n.on_click(add_n)
    nuggets_card = W.VBox([W.HBox([dd_n_size, cb_need_sauce, dd_n_sauce]),
                           W.HBox([qty_n, btn_add_n])])

    # --- 其他點心卡 ---
    dd_other = W.Dropdown(options=[(NAME_SNACK_OTHER[k], k) for k in PRICES_SNACK_OTHER.keys()],
                          value="apple_pie", description="品項")
    qty_other = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add_other = W.Button(description="加入購物車", button_style="primary", icon="plus")

    def on_other_change(change):
        if change["name"] == "value":
            set_preview(dd_other.value)
    dd_other.observe(on_other_change, names="value")
    set_preview(dd_other.value)

    def add_other(_):
        pid = dd_other.value
        name = NAME_SNACK_OTHER[pid]
        price = PRICES_SNACK_OTHER[pid]
        sku = pid
        add_to_cart(sku, name, price, qty_other.value, "snack")
        status_msg.value = f"<span style='color:green'>已加入：{name} × {qty_other.value}</span>"
    btn_add_other.on_click(add_other)
    other_card = W.VBox([W.HBox([dd_other]), W.HBox([qty_other, btn_add_other])])

    # 容器：切換顯示
    area = W.VBox()

    def rebuild_area(_=None):
        if dd_kind.value == "fries":
            area.children = [fries_card]
            set_preview(dd_fries_size.value)
        elif dd_kind.value == "nuggets":
            area.children = [nuggets_card]
            set_preview(dd_n_size.value)
        else:
            area.children = [other_card]
            set_preview(dd_other.value)

    dd_kind.observe(rebuild_area, names="value")
    rebuild_area()

    return W.VBox([dd_kind, area, status_msg])

def build_drink_card():
    dd_item = W.Dropdown(options=[(NAME_DRINK[k], k) for k in PRICES_DRINK.keys()],
                         value="coke", description="飲料")
    qty = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add = W.Button(description="加入購物車", button_style="primary", icon="plus")
    status_msg.value = ""

    def on_change(change):
        if change["name"] == "value":
            set_preview(dd_item.value)
    dd_item.observe(on_change, names="value")
    set_preview(dd_item.value)

    def _add(_):
        pid = dd_item.value
        name = NAME_DRINK[pid]
        price = PRICES_DRINK[pid]
        add_to_cart(pid, name, price, qty.value, "drink")
        status_msg.value = f"<span style='color:green'>已加入：{name} × {qty.value}</span>"

    btn_add.on_click(_add)
    return W.VBox([W.HBox([dd_item, qty, btn_add]), status_msg])

def rebuild_product_area(_=None):
    cat = tb_category.value
    if cat == "main":
        product_area.children = [build_main_card()]
    elif cat == "snack":
        product_area.children = [build_snack_card()]
    else:
        product_area.children = [build_drink_card()]

tb_category.observe(rebuild_product_area, names="value")
rebuild_product_area()

# ========= 結帳 =========
def generate_receipt(_):
    out_receipt.clear_output()

    order_id = next_order_id()
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    tax_id = txt_tax_id.value.strip()
    carrier = txt_carrier.value.strip()

    tax_line = "（未提供）"
    if tax_id:
        tax_line = tax_id + ("（統編有效）" if tw_tax_id_ok(tax_id) else "（格式需為 8 碼數字，仍可結帳）")
    carrier_line = "（未提供）"
    if carrier:
        carrier_line = carrier + ("" if carrier_hint_ok(carrier) else "（載具格式與常見型式不符，仍可結帳）")

    lines = []
    # 提示：空購物車也允許結帳，但給提醒
    if not CART and not cb_bag.value:
        lines.append("(提示) 尚未選購任何品項")
        lines.append("")

    lines.append(f"訂單編號：{order_id}")
    lines.append(f"結帳時間：{ts}")
    lines.append(f"統一編號：{tax_line}")
    lines.append(f"載具：{carrier_line}")
    lines.append("")
    lines.append("— 訂單明細 —")

    for sku, item in CART.items():
        name = item["name_display"]
        unit = item["unit_price"]
        qty = item["qty"]
        if qty == 1:
            lines.append(f"- {name} × {qty}   {currency(unit)}")
        else:
            lines.append(f"- {name} × {qty}   {currency(unit)} × {qty} = {currency(unit*qty)}")

    if cb_bag.value:
        lines.append("購物袋：$1")

    lines.append("--------------------------------")
    subtotal_wo_bag = sum(it["unit_price"] * it["qty"] for it in CART.values())
    lines.append(f"小計：{currency(subtotal_wo_bag)}")
    total = subtotal_wo_bag + (1 if cb_bag.value else 0)
    lines.append(f"總計：{currency(total)}")

    with out_receipt:
        print("\n".join(lines))

btn_checkout.on_click(generate_receipt)
cb_bag.observe(lambda ch: update_subtotal_label(), names="value")

# 初始渲染
render_cart()

# 顯示整體 UI
display(title, W.HBox([W.VBox([tb_category]), W.VBox([product_area]), W.VBox([lbl_subtotal, img_preview, W.HTML("<hr>"), W.HTML("<b>購物車</b>"), cart_area, W.HTML("<hr>"), acc_checkout])],
                      layout=W.Layout(width="100%", justify_content="space-between", align_items="flex-start", gap="24px")))

HTML(value='<h2>🍔 麥當勞點餐機（ipywidgets / PNG）</h2>')